In [28]:
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import load, Loader
import subprocess
from json import dump

## Parameters for Processing

In [29]:
output_path = Path("carbonate_run2")
raw_path = output_path / "raw_data"
actual_path = raw_path / "actual_time.yaml"
theoretical_path = raw_path / "theoretical_time.yaml"
metadata_path = output_path / "metadata.yaml"
mem_folder_path = raw_path / "memory_output"

data_dump_path = output_path / "processed_output"
output_fnames = ("actual_runtime.csv", "theoretical_runtime.csv", "bytes.csv", "all_fields.json")
MFLOPS_dict  = {"carbonate": 750, "quartz": 870, "macbook": 1250}
system = output_path.name.split('_')[0]
MFLOPS = MFLOPS_dict[system]
# MFLOPS = 751.310861 # carbonate run 1
# MFLOPS = 866.458302 # quartz run 3
# MFLOPS = 1250 # macbook run 1
FLOPS = MFLOPS * 10E6
MODELS = ["tf-necd", "tf-cod", "pytorch-qrcp", "pytorch-qr", "pytorch-svd", "pytorch-svddc", "sklearn-svddc"]

## Loading Metadata

In [30]:
with open(metadata_path, "r") as f:
    metadata = load(f, Loader=Loader)
    
metadata

{'dataset_shape': '750000000 x 10',
 'failed_regs': [],
 'failed_regs_exceptions': [],
 'repeat': 10,
 'rows_in_experiment': [10,
  31,
  100,
  316,
  1000,
  3162,
  10000,
  31622,
  100000,
  316227,
  1000000,
  3162277,
  10000000,
  31622776,
  100000000,
  316227766],
 'timer_method': 'process in nanoseconds'}

## Loading Actual Times

In [31]:
with open(actual_path, "r") as f:
    actual_times = load(f, Loader=Loader)
    
actual_times_dict = {reg_name: {row_count: np.mean([nanoseconds for rc, nanoseconds in actual_times if row_count == rc]) for row_count, _ in actual_times} for reg_name, actual_times in actual_times.items()}
actual_times_dict

{'pytorch-qr': {10: 571170.9,
  31: 471125.3,
  100: 476165.6,
  316: 490516.3,
  1000: 535655.1,
  3162: 639102.0,
  10000: 1072103.2,
  31622: 2900233.5,
  100000: 8090976.3,
  316227: 24754233.0,
  1000000: 106025172.4,
  3162277: 381256825.5,
  10000000: 1306148736.5,
  31622776: 4108885818.5,
  100000000: 13039020999.6,
  316227766: 40991094201.7},
 'pytorch-qrcp': {10: 8789374.9,
  31: 518714.7,
  100: 572675.9,
  316: 554749.6,
  1000: 511063.1,
  3162: 672673.5,
  10000: 1103546.2,
  31622: 2493751.1,
  100000: 7025731.9,
  316227: 26498797.6,
  1000000: 117504462.3,
  3162277: 418707702.8,
  10000000: 1401348732.9,
  31622776: 4415045642.2,
  100000000: 13839662326.2,
  316227766: 43476974154.2},
 'pytorch-svd': {10: 641786.9,
  31: 519715.2,
  100: 546573.7,
  316: 552327.8,
  1000: 624982.5,
  3162: 708710.3,
  10000: 1151141.5,
  31622: 2815724.9,
  100000: 7627152.4,
  316227: 25072404.4,
  1000000: 122530810.5,
  3162277: 381704429.7,
  10000000: 1295310965.8,
  31622776:

## Loading Theoretical Flop Counts

In [32]:
with open(theoretical_path, "r") as f:
    theoretical_flops = load(f, Loader=Loader)
    
theoretical_times_dict = {reg_name: {row_count: (flops / FLOPS) * 10E9 for row_count, flops in theoretical_flops} for reg_name, theoretical_flops in theoretical_flops.items()}
# theoretical_times_dict

## Loading / Processing Binary Memory Files

In [33]:
make_mempath = lambda reg_name, row_count, iter, ext: mem_folder_path / f"{'mem' if ext == 'bin' else 'memray-csv-mem'}_{reg_name}_{row_count}_{iter}.{ext}"

mem_dict = {}
for reg_name in MODELS:
    mem_dict[reg_name] = {}
    for row_count in metadata["rows_in_experiment"]:
        runs = []
        for iter in range(metadata["repeat"]):
            # use memray transform to actually create csv
            input_path = make_mempath(reg_name, row_count, iter, "bin")
            print(f"{input_path =}")
            subprocess.run(["memray", "transform", "csv", input_path, "-f"])
            
            # load csv normally, sum all mem usage
            output_path = make_mempath(reg_name, row_count, iter, "csv")
            single_mem_df = pd.read_csv(output_path, header=0, index_col=None)
            bytes_used = single_mem_df["size"].sum()
            runs.append(bytes_used)
            
        mem_dict[reg_name][row_count] = np.mean(runs)

input_path =PosixPath('carbonate_run2/raw_data/memory_output/mem_tf-necd_10_0.bin')

⚠  No symbol information was found for the Python interpreter  ⚠

Without symbolic information reports showing native traces may not accurately 
reflect stack traces. Please use an interpreter built with debug symbols for 
best results. Check https://bloomberg.github.io/memray/native_mode.html for more
information regarding how memray resolves symbols.


Wrote carbonate_run2/raw_data/memory_output/memray-csv-mem_tf-necd_10_0.csv
input_path =PosixPath('carbonate_run2/raw_data/memory_output/mem_tf-necd_10_1.bin')

⚠  No symbol information was found for the Python interpreter  ⚠

Without symbolic information reports showing native traces may not accurately 
reflect stack traces. Please use an interpreter built with debug symbols for 
best results. Check https://bloomberg.github.io/memray/native_mode.html for more
information regarding how memray resolves symbols.


Wrote carbonate_run2/raw_data/memory_out

## Aggregate and Output Data

In [34]:
json_prep_dict = {}
for reg_name in MODELS:
    json_prep_dict[reg_name] = {}
    for row_count in metadata["rows_in_experiment"]:
        json_prep_dict[reg_name][row_count] = {
            "real_runtime": actual_times_dict[reg_name][row_count],
            "theoretical_runtime": theoretical_times_dict[reg_name][row_count],
            "bytes": mem_dict[reg_name][row_count]
        }

df_real_runtime = pd.DataFrame(actual_times_dict)
df_theoretical_runtime = pd.DataFrame(theoretical_times_dict)
df_memory = pd.DataFrame(mem_dict)

data_dump_path.mkdir(exist_ok=True)

df_real_runtime.to_csv(data_dump_path / output_fnames[0], header=True, index=True)
df_theoretical_runtime.to_csv(data_dump_path / output_fnames[1], header=True, index=True)
df_memory.to_csv(data_dump_path / output_fnames[2], header=True, index=True)

with open(data_dump_path / output_fnames[3], "w") as f:
    dump(json_prep_dict, f)
    
print(f"All data has been dumped to: {data_dump_path.resolve()}")

All data has been dumped to: /Users/joshuaelms/Desktop/github_repos/Machine-Learning-Project/BetaDataExper/BigOTest/postprocessing/carbonate_run2/processed_output
